In [1]:
from pyspark.sql import *

In [2]:
import os 
os.getcwd()


Out[17]: '/databricks/driver'

In [3]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/ml/,ml/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


**READING DATA FILES AND RETURNING RDD**

In [4]:
rdd = sc.textFile('/FileStore/tables/*.bz2',minPartitions= 4)
rdd.take(4)

Out[23]: ['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 '1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA',
 '1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA',
 '1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,NA,29,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA']

**Total Number of Counts**

In [5]:
rdd.count()

Out[24]: 123534991

**Removing Header**

In [6]:
header = rdd.first()
rdd2=rdd.filter(lambda line: line != header)
rdd2.take(2)

Out[29]: ['1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA',
 '1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA']

**Implementing Mapreduce**

In [7]:
rdd1= rdd2.map(lambda x: x.split(',')[8]).map(lambda a:(a,1)).sortByKey(ascending=True).reduceByKey(lambda a,b: a+b)

**Sorting**

In [8]:
rdd2=rdd1.sortByKey()
rdd2.take(2)

Out[36]: [('9E', 521059), ('AA', 14984647)]

**Count of UniqueCarrier**

In [9]:
rdd2.take(30)

Out[37]: [('9E', 521059),
 ('AA', 14984647),
 ('AQ', 154381),
 ('AS', 2878021),
 ('B6', 811341),
 ('CO', 8145788),
 ('DH', 693047),
 ('DL', 16547870),
 ('EA', 919785),
 ('EV', 1697172),
 ('F9', 336958),
 ('FL', 1265138),
 ('HA', 274265),
 ('HP', 3636682),
 ('ML (1)', 70622),
 ('MQ', 3954895),
 ('NW', 10292627),
 ('OH', 1464176),
 ('OO', 3090853),
 ('PA (1)', 316167),
 ('PI', 873957),
 ('PS', 83617),
 ('TW', 3757747),
 ('TZ', 208420),
 ('UA', 13299817),
 ('US', 14075530),
 ('WN', 15976022),
 ('XE', 2350309),
 ('YV', 854056)]

**Reading data in dataframe (Alternative method)**

In [10]:
DF = spark.read.csv('/FileStore/tables/*.bz2', header="true", inferSchema="true")

**Total counts by UniqueCarrier**

In [11]:
sorted(DF.groupBy(['UniqueCarrier']).count().collect())

Out[55]: [Row(UniqueCarrier='9E', count=521059),
 Row(UniqueCarrier='AA', count=14984647),
 Row(UniqueCarrier='AQ', count=154381),
 Row(UniqueCarrier='AS', count=2878021),
 Row(UniqueCarrier='B6', count=811341),
 Row(UniqueCarrier='CO', count=8145788),
 Row(UniqueCarrier='DH', count=693047),
 Row(UniqueCarrier='DL', count=16547870),
 Row(UniqueCarrier='EA', count=919785),
 Row(UniqueCarrier='EV', count=1697172),
 Row(UniqueCarrier='F9', count=336958),
 Row(UniqueCarrier='FL', count=1265138),
 Row(UniqueCarrier='HA', count=274265),
 Row(UniqueCarrier='HP', count=3636682),
 Row(UniqueCarrier='ML (1)', count=70622),
 Row(UniqueCarrier='MQ', count=3954895),
 Row(UniqueCarrier='NW', count=10292627),
 Row(UniqueCarrier='OH', count=1464176),
 Row(UniqueCarrier='OO', count=3090853),
 Row(UniqueCarrier='PA (1)', count=316167),
 Row(UniqueCarrier='PI', count=873957),
 Row(UniqueCarrier='PS', count=83617),
 Row(UniqueCarrier='TW', count=3757747),
 Row(UniqueCarrier='TZ', count=208420),
 Row(UniqueCarrier='UA', count=13299817),
 Row(UniqueCarrier='US', count=14075530),
 Row(UniqueCarrier='WN', count=15976022),
 Row(UniqueCarrier='XE', count=2350309),
 Row(UniqueCarrier='YV', count=854056)]